In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import NuLattice.smearedInteraction as smearI
import NuLattice.CCM.coupled_cluster as ccm
import NuLattice.references as ref
import NuLattice.FCI.few_body_diagonalization as fbd
import NuLattice.lattice as lat
from scipy.sparse.linalg import eigsh as arpack_eigsh
import NuLattice.constants as consts
from opt_einsum import contract
import scipy.sparse as sparse

In [5]:
myL= 4
bpi = 0.7
sL = 0.08
sNL = 0.08
a = 1.0 / 100.0
at = 1.0 / 150.0
c0 = -1.85e-05/ (a ** 2)
a_lat = consts.hbarc * a
lattice = lat.get_lattice(myL)
tKin = smearI.tKin(myL, 3, a)
holes = ref.ref_2H_gs

In [11]:
inter = smearI.shortRangeV(lattice,myL, sL, sNL, c0)
print(len(inter))

Densities Generated
rho*f_SL Generated
Compressing interaction... Compressed
Compressing interaction... Compressed
Compressing interaction... Compressed
Compressing interaction... Compressed
Converting to List
Full Interaction Generated
10519132


In [12]:
np.savez_compressed('shortRangeV_L=4_sl=08_snl=08.npz', inter)
del(inter)

In [15]:
inter = np.load('shortRangeV_L=4_sl=08_snl=08.npz')['arr_0']

In [17]:
# Compute the deuteron
# additive quantum numbers
numpart=2 # number of nucleons
tz = 0    # twice the value of isospin projection
sz = 2    # twice the value of spin projection
my_basis = lat.get_sp_basis(myL)
nstat =  len(my_basis)

# get two-body basis as a dictionary for lookup
H2_lookup = fbd.get_many_body_states(my_basis, numpart, total_tz=tz, total_sz=sz)
print("matrix dimension:", len(H2_lookup))

# make scipy.sparse.csr_matrix for kinetic energy 
T2_csr_mat = fbd.get_csr_matrix_scalar_op(H2_lookup, tKin, nstat)
print("kinetic energy done")

# make scipy.sparse.csr_matrix for 2-body interactions 
V2_csr_mat = fbd.get_csr_matrix_scalar_op(H2_lookup, inter, nstat)
print("2-body interaction done")

# add all into Hamiltonian
H2_csr_mat = T2_csr_mat + V2_csr_mat

# compute lowest eigenvalue(s)
k_eig=10 # number of eigenvalues
vals, vecs = arpack_eigsh(H2_csr_mat, k=k_eig, which='SA')
print("Energies (MeV):", vals / a)

matrix dimension: 4096
kinetic energy done
2-body interaction done
Energies (MeV): [-5.17260863  5.95401204  5.95401204  5.95401204  5.95401204  5.95401204
  5.95401204 12.36918577 12.36918577 12.36918577]
